In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata
import itertools
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
api_version="2023-12-01-preview"
engine = 'gpt-4o-mini'
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY_GPT4OMINI"),  
    api_version=api_version,
    azure_endpoint = os.getenv("OPENAI_API_BASE_GPT4OMINI")
    )

data = pd.read_csv('../data.csv')

In [34]:
gold_sd = dict(data.query('flag =="test"')[['question_number', 'table_id']].values)

In [2]:
class make_content():
    def __init__(self, path):
        # 各doc_idのパスを指定
        self.path = path
        self.doc_id = path.split('/')[-1]
    
    def edit_text(self, text):
        return re.sub(r'[^\u3040-\u309F\u30A0-\u30FF\uFF65-\uFF9F\u4E00-\u9FFF]', '', text)
    
    def make_content(self):
        try:
            with open(f"../data/contents/{self.doc_id}.pkl", 'rb') as f:
                dct = pickle.load(f)
                f.close()
            return dct
        except:
            dct = pd.DataFrame()
            files = os.listdir(self.path)
            tmp_h1 = ''
            tmp_h2 = ''
            tmp_h3 = ''
            tmp_h4 = ''
            tmp_h5 = ''
            tmp_h6 = ''
            if not os.path.exists(f'../data/{self.doc_id}'):
                os.makedirs(f'../data/{self.doc_id}')
            for file in files:
                with open(f'{self.path}/{file}', 'r') as f:
                    soup = bs(f, 'html.parser')
                    f.close()
                html_tag = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'table'])
                for index, tag in enumerate(html_tag):
                    if tag.name == 'h1':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h1 = self.edit_text(text)
                        tmp_h2 = ''
                        tmp_h3 = ''
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h2':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h2 = self.edit_text(text)
                        tmp_h3 = ''
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h3':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h3 = self.edit_text(text)
                        tmp_h4 = ''
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h4':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h4 = self.edit_text(text)
                        tmp_h5 = ''
                        tmp_h6 = ''
                    elif tag.name == 'h5':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h5 = self.edit_text(text)
                        tmp_h6 = ''
                    elif tag.name == 'h6':
                        text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                        if text == '':
                            continue
                        tmp_h6 = self.edit_text(text)
                    elif tag.name == 'table':
                        dct =pd.concat([dct, pd.DataFrame({'h1': [tmp_h1], 'h2': [tmp_h2], 'h3': [tmp_h3], 'h4': [tmp_h4], 'h5': [tmp_h5], 'h6': [tmp_h6], 'table_id':[tag.get('table-id')],'html': [file]})])
            pickle.dump(dct, open(f"../data/contents/{self.doc_id}.pkl", 'wb'))
            return dct

In [ ]:
"""
for _, rows in data.iterrows():
    path = f'../{rows["flag"]}/reports/{rows["doc_id"]}'
    make_content(path).make_content()
"""

In [45]:
class search_table():
    def __init__(self, words, doc_id, df_train, question):
        self.words = words
        self.doc_id = doc_id
        self.df_train = df_train
        self.question = question

    # 類似質問から目次データを取得
    def get_contents(self, ):
        #重複単語数でランク付けしたほうがいいかもしれない
        f = lambda x:sum([(word in x) for word in self.words]) 
        similary_tables = self.df_train[self.df_train['words'].apply(f) == self.df_train['words'].apply(f).max()]['table_id'].values
        
        df_contents = pd.DataFrame()
        for table_id in similary_tables:
            doc_id = table_id.split('-')[0]
            with open(f"../data/contents/{doc_id}.pkl", 'rb') as f:
                dct = pickle.load(f)
                f.close()
            df_contents = pd.concat([df_contents, dct.query('table_id == @table_id')], axis=0)

        return df_contents.drop_duplicates(subset=['h1', 'h2', 'h3', 'h4', 'h5'])
    
    #類似質問の目次データを該当文書のtable_id候補を取得する
    def get_tables(self,):
        contents = self.get_contents()
        contents.reset_index(inplace=True)
        with open(f"../data/contents/{self.doc_id}.pkl", 'rb') as f:
            myself_contents = pickle.load(f)
            f.close()
            
        if contents.empty:
            return myself_contents['table_id'].values
    
        tables_dict= {}
        for index, rows in contents.iterrows():
            tmp = myself_contents
            tmp_header = 'h0'
            if rows['h1'] != '' and not myself_contents.query('h1 == @rows["h1"]').empty:
                tmp = myself_contents.query('h1 == @rows["h1"]',)
                tmp_header = 'h1'
                
            if rows['h2'] != '' and not myself_contents.query('h2 == @rows["h2"]').empty:
                tmp = myself_contents.query('h2 == @rows["h2"]',)
                tmp_header = 'h2'

            if rows['h3'] != '' and not myself_contents.query('h3 == @rows["h3"] or h4 == @rows["h3"]').empty:
                tmp = myself_contents.query('h3 == @rows["h3"] or h4 == @rows["h3"]')
                tmp_header = 'h3'
            
            if rows['h4'] != '' and not myself_contents.query('h4 == @rows["h4"] or h5 == @rows["h4"]').empty:
                tmp = myself_contents.query('h4 == @rows["h4"] or h5 == @rows["h4"]')
                tmp_header = 'h4'

            if rows['h5'] != '' and not myself_contents.query('h5 == @rows["h5"] or h4 == @rows["h5"]').empty:
                tmp = myself_contents.query('h5 == @rows["h5"] or h4 == @rows["h5"]',)
                tmp_header = 'h5'
            
            tables_dict[f'no{index}'] = {'depth':tmp_header, 'tables':tmp['table_id'].values}
    
        max_depth = max(d['depth'] for d in tables_dict.values())
        tables = [d['tables'] for d in tables_dict.values() if d['depth'] == max_depth]
        tables = list(set(itertools.chain.from_iterable(tables)))
        return tables

    def answer_table(self):
        targets_tables = self.get_tables()
        if len(targets_tables) == 1:
            return targets_tables 
        else:
            tables_score = {}
            text = ''
            for table in targets_tables:
                html = table.split('-')[1]
                if os.path.exists(f'../data/{self.doc_id}/{table}.pickle'):                   
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'r',) as f:
                        text = f.read()
                        f.close()
                else:
                    with open(f"../test/reports/{self.doc_id}/{html}.html", 'rb') as f:
                        soup = bs(f, 'html.parser')
                        f.close()
                    tags = soup.find_all(['table', 'p', 'h3', 'h4'])
                    for tag in tags:
                        if tag.name == 'table':
                            if tag.get('table-id') == table:
                                text += unicodedata.normalize('NFKC', tag.get_text()).replace(' ', '').replace('\n', '')
                                break
                            else:
                                text = ''
                        else:
                            if tag.parent.name != 'td':
                                text += unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'w', encoding='utf-8') as f2:
                        f2.write(text)
                        f2.close()
                tables_score[table] = sum([(word in text) for word in self.words])
            print(tables_score)
            return [kv[0] for kv in tables_score.items() if kv[1] == max(tables_score.values())]

    #ChatGPTに質問を投げて、表から回答を抽出する
    def answer(self, ):
        tables = self.answer_table()
        if len(tables) == 1:
            return tables[0]
        
        else:
            table_text = ''
            for i, table in enumerate(tables):
                if os.path.exists(f'../data/{self.doc_id}/{table}.pickle'):                   
                    with open(f'../data/{self.doc_id}/{table}.pickle', 'r',) as f:
                        table_text +=f"""## Table{i}\n{f.read()}\n"""
                        f.close()
        
            prompt =f"質問に回答が記載されている表の番号を教えてください。\n##question\n{self.question}\n{table_text}\n##Output\nTable_num:<Table_num>"
            
            messages=[
                {"role": "user", "content": prompt}, 
                ]
            response = client.chat.completions.create(
            model= engine,
            messages = messages,
            temperature=0,
            max_tokens=2048,
            top_p=0.1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
            )
            return response.choices[0].message.content        

In [52]:
df_train = data.query('flag in ["train", "valid"]')
question, words, doc_id, table_id = data.query('question_number =="question_test34"')[['question',"words","doc_id", 'table_id']].values[0]
words = json.loads(words.replace("'", '"'))
print(words)
print(table_id)
print(search_table(words, doc_id, df_train, question).answer_table())

['研究開発費', '販売費及び一般管理費']
S100RH38-0105010-tab99
{'S100RH38-0105010-tab141': 0, 'S100RH38-0105010-tab163': 0, 'S100RH38-0105010-tab121': 0, 'S100RH38-0105010-tab146': 0, 'S100RH38-0105020-tab206': 0, 'S100RH38-0105010-tab165': 0, 'S100RH38-0105010-tab193': 0, 'S100RH38-0105020-tab224': 0, 'S100RH38-0105010-tab152': 0, 'S100RH38-0105010-tab175': 0, 'S100RH38-0105010-tab91': 0, 'S100RH38-0105010-tab142': 0, 'S100RH38-0105010-tab94': 0, 'S100RH38-0105010-tab137': 0, 'S100RH38-0105010-tab179': 0, 'S100RH38-0105010-tab122': 0, 'S100RH38-0105010-tab132': 0, 'S100RH38-0105020-tab220': 0, 'S100RH38-0105010-tab148': 0, 'S100RH38-0105020-tab223': 0, 'S100RH38-0105010-tab89': 0, 'S100RH38-0105010-tab149': 0, 'S100RH38-0105010-tab183': 0, 'S100RH38-0105010-tab176': 0, 'S100RH38-0105010-tab138': 0, 'S100RH38-0105020-tab214': 0, 'S100RH38-0105010-tab128': 0, 'S100RH38-0105010-tab171': 0, 'S100RH38-0105010-tab101': 0, 'S100RH38-0105010-tab102': 0, 'S100RH38-0105010-tab133': 0, 'S100RH38-0105010-tab156'

In [31]:
from tqdm import tqdm

df_train = data.query('flag in ["train", "valid"]')
answer_sheet = {}
for index, row in data.query('flag =="test"')[:20].iterrows():
    words, doc_id, table_id = row[["words","doc_id", 'table_id']]
    words = json.loads(words.replace("'", '"'))
    answer = search_table(words, doc_id, df_train).answer_table()
    answer_sheet[row['question_number']] = answer

In [35]:
total = 0
correct = 0

for key in answer_sheet.keys():
    total += 1
    if gold_sd[key] in answer_sheet[key]:
        correct += 1
    else:
        print(f'{key}, TRUE:{gold_sd[key]}, PRED:{answer_sheet[key]}')
        pass
        
print(correct/total)

question_test33, TRUE:S100RH38-0105010-tab99, PRED:['S100RH38-0105010-tab98']
question_test34, TRUE:S100RH38-0105010-tab99, PRED:['S100RH38-0105010-tab98']
0.9


In [33]:
#json.dump(answer_sheet,open('../test/answersheet.json', 'w') ,indent=0)